In [1]:
!which python

/Users/ahmadabdullahtariq/Documents/Projects/Thesis/.venv/bin/python


In [1]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go

In [2]:
df = pd.read_pickle('../dataset/preprocessed/284_imputed_renamed.pkl')

In [3]:
print(df.isna().sum())
print(df.dtypes)
df.describe(include='all')

DOA              0
LeadTime         0
DOW              0
Price            0
TotalCapacity    0
Reservations     0
dtype: int64
DOA              datetime64[ns]
LeadTime                  int64
DOW                       int64
Price                   float64
TotalCapacity             int64
Reservations              int64
dtype: object


,DOA,LeadTime,DOW,Price,TotalCapacity,Reservations
count,60387,60387.000000,60387.000000,60387.000000,60387.000000,60387.000000
unique,719,NaN,NaN,NaN,NaN,NaN
top,2019-02-04 00:00:00,NaN,NaN,NaN,NaN,NaN
freq,90,NaN,NaN,NaN,NaN,NaN
first,2018-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
last,2019-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,43.568020,2.999851,77.716630,289.577392,129.447447
std,NaN,26.012538,2.003074,39.163356,11.062532,71.519319
min,NaN,0.000000,0.000000,30.240000,0.000000,0.000000
25%,NaN,21.000000,1.000000,53.820000,290.000000,72.000000


# Pick Up Model 
**Idea:** Classicial model also known as advance booking models. They use development of bookings over time for stay date to predict final date. The key concept here is how much the value has increased over time, and this gives us the trend for increments over time for prediction.
<br><br>
**Types**: There are two types of pick-up models.<br>
1. Additive <br>
2. Multiplicative <br>

## Additive
\begin{gather*} 
ROH_{0,t} = ROH_{today} + P_{i,t}\\
P_{i,t} = \frac{\sum_{i=1}^{n}{True_t-ROH_{0,t}}}{n}
\end{gather*}

In [4]:
df.head()

,DOA,LeadTime,DOW,Price,TotalCapacity,Reservations
0,2018-01-01,88,0,76.05,290,58
1,2018-01-01,87,0,76.05,290,58
2,2018-01-01,85,0,76.05,290,58
3,2018-01-01,81,0,76.05,290,63
4,2018-01-01,80,0,76.05,290,62


In [5]:
df = df.pivot(index='DOA',columns='LeadTime',values='Reservations')
df = df.ffill(axis=1).bfill(axis=1)
# Consider only 2018 and 2019
df = df.iloc[:719]

In [6]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
DOA,,,,,,,,,,,,,,,,,,,,,
2018-01-01,209.0,206.0,206.0,198.0,204.0,198.0,196.0,199.0,197.0,189.0,...,62.0,63.0,63.0,63.0,63.0,58.0,58.0,58.0,58.0,58.0
2018-01-02,223.0,214.0,209.0,209.0,201.0,203.0,188.0,183.0,184.0,185.0,...,52.0,50.0,51.0,51.0,51.0,51.0,49.0,49.0,48.0,48.0
2018-01-03,238.0,218.0,212.0,207.0,207.0,194.0,195.0,174.0,168.0,169.0,...,40.0,40.0,39.0,39.0,39.0,39.0,39.0,36.0,36.0,36.0
2018-01-04,234.0,222.0,211.0,208.0,203.0,203.0,189.0,186.0,161.0,152.0,...,35.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,32.0,32.0
2018-01-05,261.0,238.0,223.0,212.0,210.0,207.0,207.0,194.0,185.0,158.0,...,29.0,29.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,26.0


In [7]:
dx = df.copy()
for col in range(0,89):
    df[col] = dx[col]-dx[col+1]

In [8]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
DOA,,,,,,,,,,,,,,,,,,,,,
2018-01-01,3.0,0.0,8.0,-6.0,6.0,2.0,-3.0,2.0,8.0,-1.0,...,-1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,58.0
2018-01-02,9.0,5.0,0.0,8.0,-2.0,15.0,5.0,-1.0,-1.0,8.0,...,2.0,-1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,48.0
2018-01-03,20.0,6.0,5.0,0.0,13.0,-1.0,21.0,6.0,-1.0,-2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,36.0
2018-01-04,12.0,11.0,3.0,5.0,0.0,14.0,3.0,25.0,9.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,32.0
2018-01-05,23.0,15.0,11.0,2.0,3.0,0.0,13.0,9.0,27.0,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.0


In [9]:
print("Target Date 2018-01-01 final occupancy:",np.sum(df.iloc[0].values))

Target Date 2018-01-01 final occupancy: 209.0


In [35]:
train = df.iloc[:575]
test = df.iloc[575:]

In [36]:
# sum all columns and average
p = train.sum(axis = 0)/575

In [37]:
np.sum(p.values)

91.5097391304348

LeadTime
0     3.886957
1     4.208696
2     4.029565
3     4.638261
4     4.822609
        ...   
83    0.793043
84    0.796522
85    0.812174
86    0.794783
87    0.761739
Length: 88, dtype: float64

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=p.index, y=p[:88],mode='lines',name='Avg Lead Time'))
fig.show()   

## Multiplicative

In [18]:
from operator import mul
from functools import reduce

In [19]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
DOA,,,,,,,,,,,,,,,,,,,,,
2018-01-01,3.0,0.0,8.0,-6.0,6.0,2.0,-3.0,2.0,8.0,-1.0,...,-1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,58.0
2018-01-02,9.0,5.0,0.0,8.0,-2.0,15.0,5.0,-1.0,-1.0,8.0,...,2.0,-1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,48.0
2018-01-03,20.0,6.0,5.0,0.0,13.0,-1.0,21.0,6.0,-1.0,-2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,36.0
2018-01-04,12.0,11.0,3.0,5.0,0.0,14.0,3.0,25.0,9.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,32.0
2018-01-05,23.0,15.0,11.0,2.0,3.0,0.0,13.0,9.0,27.0,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.0


In [20]:
dx = df.copy()
for col in range(0,89):
    df[col] = dx[col]/dx[col+1]

In [21]:
reduce(mul, df.iloc[0].values)

nan